# Text Summarization Overview

In order to perform the feature extraction, I created a `class TextSummarizer` which serves both as the feature extractor and the text summarizer. The class has the following functions and their corresponding capabilities:
* `avg_word`: Function for getting the average length of a word in a text
* `count_punctuation`: Function for getting the number of punctuations in a text
* `get_optimal_number_sentences`: Function for getting the optimal number of sentences for extractive summarization
* `extract_text_features`: Function which returns the following features number of stopwords, punctuations, numerical characters, words, average word length and stopwords to word ratio.
* `extractive_summarizer`: Function for performing extractive text summarization

# Building the TextSummarizer class

In [ ]:
!pip install sentencepiece
!pip install transformers
!pip install tensorflow-gpu # For CPMTokenizer
!pip install bert-extractive-summarizer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.6 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
import pandas as pd
import numpy as np

# Data preprocessing
import string
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

# Text Summarization
from transformers import *
from summarizer import Summarizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
/usr/local/lib/python3.10/dist-packages/transformers/deepspeed.py:23: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation_tf_utils.py:24: FutureWarning: Importing `TFGenerationMixin` from `src/transformers/generation_tf_uti

In [ ]:
class TextSummarizer:
  def __init__(self, data):
    self.data = data

  # Helper functions

  # Get average word length in a document
  def avg_word(self, data):
    words = data.split()
    length = (sum(len(word) for word in words)/(len(words)+0.000001))

    return length

  # Get number of punctuations in a document
  def count_punctuation(self, data):
    punctuation_count = sum([1 for char in data if char in string.punctuation])

    return punctuation_count

  # Get optimal number of sentences for extractive summarization
  def get_optimal_number_sentences(self, data, model):

    optimal_num_sentences = model.calculate_optimal_k(data, k_max=10)

    return optimal_num_sentences

  # Extract numerical text features
  def extract_text_features(self, text_column):

    """
    Extracts text features such as number of stopwords, punctuations,
    numerical characters, average word length, average document length
    :param text_column: dataframe column to perform feature extraction on
    :return: dataframe with new feature columns
    """

    # Get number of stop words
    stop_words = stopwords.words('english')
    self.data["num_stopwords"] = self.data[text_column].apply(lambda x:
    len([x for x in x.split() if x in stop_words]))

    # Get number of punctuations
    self.data["num_punctuations"] = self.data[text_column].apply(lambda x:
    self.count_punctuation(x))

    # Get number of numerical characters
    self.data["num_numerics"] = self.data[text_column].apply(lambda x:
    len([x for x in x.split() if x.isdigit()]))

    # Get number of words in the document
    self.data["num_words"] = self.data[text_column].apply(lambda x:
    len(str(x).split(" ")))

    # Get average word length in document

    self.data["avg_word_length"] = self.data[text_column].apply(lambda x:
    round(self.avg_word(x),1))

    # Get the stopwords to word ratio
    self.data["stopwords_to_words_ratio"] = round(self.data["num_stopwords"] / self.data["num_words"], 3)

    return self.data

  def extractive_summarizer(self, model, text_column):

    """
    Performs extractive text summarization with BERT and allows for different
    pretrained model loading and configurations.
    :param model: initialized pretrained model
    :param text_column: dataframe column to perform text_summarization on
    :return: dataframe with summarized text columns
    """

    self.data["extractive_summarized_text"] = self.data[text_column].apply(lambda x:
    "".join(model(x, num_sentences=self.get_optimal_number_sentences(x, model))))

    return self.data

# Performing Text Summarization

## Extractive Summarization

In [ ]:
text_df = pd.read_csv("subtitles_cleaned.csv")
text_df.drop("Unnamed: 0", axis=1, inplace=True)
text_class = TextSummarizer(text_df)

text_class.extract_text_features("full_text")

# Use scibert to perform extractive summarization
pretrained_model = 'allenai/scibert_scivocab_uncased'

# Load model, model config and tokenizer via Transformers
custom_config = AutoConfig.from_pretrained(pretrained_model)
custom_config.output_hidden_states=True
custom_tokenizer = AutoTokenizer.from_pretrained(pretrained_model)
custom_model = AutoModel.from_pretrained(pretrained_model, config=custom_config)

# Create pretrained-model object
model = Summarizer(custom_model=custom_model, custom_tokenizer=custom_tokenizer)

# Extractive summarization
extractive_summarized_text = text_class.extractive_summarizer(model, "full_text")

# Optional: Save dataframe containing extractive summaries
extractive_summarized_text.to_csv("extractive_summarized_dataframe_final.csv")

# Check extractive summaries
extractive_summaries = extractive_summarized_text["extractive_summarized_text"]
print(extractive_summaries)

https://huggingface.co/allenai/scibert_scivocab_uncased/resolve/main/config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpa002tp2s


storing https://huggingface.co/allenai/scibert_scivocab_uncased/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/858852fd2471ce39075378592ddc87f5a6551e64c6825d1b92c8dab9318e0fc3.03ff9e9f998b9a9d40647a2148a202e3fb3d568dc0f170dda9dda194bab4d5dd
creating metadata file for /root/.cache/huggingface/transformers/858852fd2471ce39075378592ddc87f5a6551e64c6825d1b92c8dab9318e0fc3.03ff9e9f998b9a9d40647a2148a202e3fb3d568dc0f170dda9dda194bab4d5dd
loading configuration file https://huggingface.co/allenai/scibert_scivocab_uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/858852fd2471ce39075378592ddc87f5a6551e64c6825d1b92c8dab9318e0fc3.03ff9e9f998b9a9d40647a2148a202e3fb3d568dc0f170dda9dda194bab4d5dd
Model config BertConfig {
  "_name_or_path": "allenai/scibert_scivocab_uncased",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range

storing https://huggingface.co/allenai/scibert_scivocab_uncased/resolve/main/vocab.txt in cache at /root/.cache/huggingface/transformers/33593020f507d72099bd84ea6cd2296feb424fecd62d4a8edcc2a02899af6e29.38339d84e6e392addd730fd85fae32652c4cc7c5423633d6fa73e5f7937bbc38
creating metadata file for /root/.cache/huggingface/transformers/33593020f507d72099bd84ea6cd2296feb424fecd62d4a8edcc2a02899af6e29.38339d84e6e392addd730fd85fae32652c4cc7c5423633d6fa73e5f7937bbc38
loading file https://huggingface.co/allenai/scibert_scivocab_uncased/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/33593020f507d72099bd84ea6cd2296feb424fecd62d4a8edcc2a02899af6e29.38339d84e6e392addd730fd85fae32652c4cc7c5423633d6fa73e5f7937bbc38
loading file https://huggingface.co/allenai/scibert_scivocab_uncased/resolve/main/tokenizer.json from cache at None
loading file https://huggingface.co/allenai/scibert_scivocab_uncased/resolve/main/added_tokens.json from cache at None
loading file https://huggingf

storing https://huggingface.co/allenai/scibert_scivocab_uncased/resolve/main/pytorch_model.bin in cache at /root/.cache/huggingface/transformers/de14937a851e8180a2bc5660c0041d385f8a0c62b1b2ccafa46df31043a2390c.74830bb01a0ffcdeaed8be9916312726d0c4cd364ac6fc15b375f789eaff4cbb
creating metadata file for /root/.cache/huggingface/transformers/de14937a851e8180a2bc5660c0041d385f8a0c62b1b2ccafa46df31043a2390c.74830bb01a0ffcdeaed8be9916312726d0c4cd364ac6fc15b375f789eaff4cbb
loading weights file https://huggingface.co/allenai/scibert_scivocab_uncased/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/de14937a851e8180a2bc5660c0041d385f8a0c62b1b2ccafa46df31043a2390c.74830bb01a0ffcdeaed8be9916312726d0c4cd364ac6fc15b375f789eaff4cbb
Some weights of the model checkpoint at allenai/scibert_scivocab_uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictio

0    Word lattice decoding has proven useful in spo...
1    We formulate the problem of nonprojective depe...
2    We propose a cascaded linear model for joint C...
3    In this paper, we propose a novel string-todep...
4    We present a novel transition system for depen...
Name: extractive_summarized_text, dtype: object
